In [32]:
#export
from export.nb_00 import *
from torch import nn, optim, tensor, Tensor, hub
import torch.nn.functional as F
from torch.utils.data import DataLoader, SequentialSampler, RandomSampler
import gzip, pickle

from functools import partial
from typing import Any, Collection, Callable, NewType, List, Union, TypeVar, Optional, Generator, Iterable

In [8]:
#export

mnist_path=Path(r'd:\datasets\data\mnist.pkl.gz')
# mnist_path.ls()

In [27]:
#export
def get_mnist():
    with gzip.open(mnist_path, 'rb') as f:
        ((x_train, y_train),(x_valid, y_valid), _)=pickle.load(f, encoding='latin-1')
    x_train, y_train, x_valid, y_valid= map(tensor, (x_train, y_train, x_valid, y_valid))
    return (x_train, y_train), (x_valid, y_valid)

The most important argument of DataLoader constructor is dataset, which indicates a dataset object to load data from. PyTorch supports two different types of datasets:

    * `map style`
    * `iter style`

In [33]:
class Dataset():
    def __init__(self, x, y):
        self.x, self.y=x, y
        
    def __getitem__(self, i): return self.x[i], self.y[i]
    def __len__(self): return len(self.x)

In [34]:
(x_train, y_train), (x_valid, y_valid)= get_mnist()
train_ds, valid_ds= Dataset(x_train, y_train), Dataset(x_valid, y_valid)

In [36]:
bs=128
epochs=3

In [37]:
def collate(b):
    xs,ys = zip(*b)
    return torch.stack(xs),torch.stack(ys)

train_dl = DataLoader(train_ds, bs, sampler=RandomSampler(train_ds), collate_fn=collate)
valid_dl = DataLoader(valid_ds, bs, sampler=SequentialSampler(valid_ds), collate_fn=collate)

In [43]:
Rank0Tensor = NewType('OneEltTensor', Tensor)
LossFunction = Callable[[Tensor, Tensor], Rank0Tensor]
Model = nn.Module

In [45]:
def fit(epochs, model, loss_func, opt, train_dl, valid_dl):
    for epoch in range(epochs):
        # Handle batchnorm / dropout
        model.train()
#         print(model.training)
        for xb,yb in train_dl:
            loss = loss_func(model(xb), yb)
            loss.backward()
            opt.step()
            opt.zero_grad()

        model.eval()
#         print(model.training)
        with torch.no_grad():
            tot_loss,tot_acc = 0.,0.
            for xb,yb in valid_dl:
                pred = model(xb)
                tot_loss += loss_func(pred, yb)
                tot_acc  += accuracy (pred,yb)
        nv = len(valid_dl)
        print(epoch, tot_loss/nv, tot_acc/nv)
    return tot_loss/nv, tot_acc/nv

In [56]:
class Lin_Block(nn.Module):
    def __init__(self, inp, out):
        super().__init__()
        self.lin=nn.Linear(inp, out)
        
    def forward(self, x):
        return nn.ReLU(self.lin(x), 0)

class MLP(nn.Module):
    def __init__(self, layers:list):
        super().__init__()
        self.m1=nn.ModuleList([Lin_Block(layers[i], layers[i+1]) for i in range(len(layers)-1)])
        self.m2=nn.ModuleList([self.m1, nn.Linear(layers[-1], 10)])
        
    def forward(self, x):
        x=self.m1(x)
        return sigmoid(self.m2(x))

In [57]:
layers=[28*28, 512, 256, 128, 64]
model=MLP(layers)

In [58]:
y=model(x_train)

NotImplementedError: 